# Corte de fatores de risco para malformacoes cardiacas SINAC

In [34]:
from pysus import SINASC
import pandas as pd
import re

In [35]:
# Lista de anos desejados
sinasc = SINASC().load()

anos = [2022,2023]

files = sinasc.get_files("DN", year=anos)

In [36]:
parquets = sinasc.download(files)
dfs = [p.to_dataframe() for p in parquets]
df = pd.concat(dfs, ignore_index=True)

1127918it [00:00, 12045891.06it/s]    


In [37]:
#print(df.columns)
colunas_numericas = ["TPNASCASSI","IDADEMAE", "ESTCIVMAE", "GRAVIDEZ","PESO", "SEMAGESTAC", "CONSULTAS","PARTO","CODOCUPMAE","QTDFILVIVO","QTDFILMORT"]
for col in colunas_numericas:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"Numero total de Nacidos vivos registrados: {len(df)}")

Numero total de Nacidos vivos registrados: 2561922


## Filtrar df

In [38]:
variaveis = ["IDADEMAE", "ESTCIVMAE", "GRAVIDEZ","PESO", "SEMAGESTAC", "CONSULTAS", "CODANOMAL","ESCMAE2010","PARTO","CODOCUPMAE","SEXO","QTDFILVIVO","QTDFILMORT"]
df_filtrado_variaveis = df[variaveis]

In [39]:
df =0

In [42]:
def filter_ignored_data(df):
    """
    Remove do DataFrame as linhas onde alguma das variáveis
    ESTCIVMAE, GRAVIDEZ, ESCMAE2010 ou PARTO tenha o valor 9.
    
    Parâmetros:
      df: DataFrame contendo as colunas mencionadas.
      
    Retorna:
      DataFrame filtrado.
    """
    mask = (
        (df['ESTCIVMAE'] != 9) &
        (df['GRAVIDEZ'] != 9) &
        (df['ESCMAE2010'] != 9) &
        (df['PARTO'] != 9) &
        (df['SEXO'] != 0)&
        (df['CONSULTAS'] !=9)&
        (df['QTDFILMORT'] != 99)
    )
    df_filtrado = df[mask].copy()
    return df_filtrado


In [43]:
df_filtrado_variaveis = filter_ignored_data(df_filtrado_variaveis)

print(f"Numero total de Nacidos vivos registrados depois da exclusao de dados faltantes das variaveis SEXO, Tipo de PARTO, GRAVIDEZ,Consultas, EStado Civil e ESCMAE2010: {len(df_filtrado_variaveis)}")

Numero total de Nacidos vivos registrados depois da exclusao de dados faltantes das variaveis SEXO, Tipo de PARTO, GRAVIDEZ,Consultas, EStado Civil e ESCMAE2010: 2464019


In [ ]:
# Supondo que seu DataFrame com as variáveis filtradas seja df_filtrado_variaveis
# Primeiro, padronize a coluna de códigos\
df_filtrado_variaveis.loc[:, 'CODANOMAL'] = (
    df_filtrado_variaveis['CODANOMAL']
    .astype(str)
    .str.strip()
    .str.upper()
)

df_filtrado_variaveis = df_filtrado_variaveis.dropna(subset=['ESCMAE2010', 'PARTO', 'CODOCUPMAE', 'ESTCIVMAE','GRAVIDEZ'])

# Defina os códigos de deformidades cardíacas
codigos_cid = [
    "Q20",  # Malformações congênitas das câmaras e das comunicações cardíacas
    "Q21",  # Malformações congênitas dos septos cardíacos
    "Q22",  # Malformações congênitas das valvas pulmonar e tricúspide
    "Q23",  # Malformações congênitas das valvas mitral e aórtica
    "Q24",  # Outras malformações cardíacas congenítas
    "Q25",  # Anomalias dos grandes vasos arteriais (ex: transposição)
    "Q26",  # Anomalias arteriovenosas cerebrais não classificadas em outra parte.
    "Q27",  # Outras anomalias dos vasos sanguíneos periféricos.
    "Q28"   # Outras malformações congenitas do aparelho circulatório
]


# Crie uma expressão regular que verifique se o valor começa com algum dos códigos
regex_cid = '^(' + '|'.join(codigos_cid) + ')'

# 3. Crie as máscaras:
# Registros sem anomalia: onde CODANOMAL é uma string vazia
mask_sem_anomalia = (df_filtrado_variaveis['CODANOMAL'] == '')

# Registros com deformidades cardíacas: onde CODANOMAL contém (ou inicia com) um dos códigos
mask_cardiacas = df_filtrado_variaveis['CODANOMAL'].str.contains(regex_cid, na=False)

# Filtre os registros que tenham ou nenhuma anomalia ou deformidades cardíacas
df_model = df_filtrado_variaveis[mask_sem_anomalia | mask_cardiacas].copy()

# 4. Crie a variável binária 'CARDIAC': 1 para deformidade, 0 para ausência
df_model['CARDIAC'] = 0
df_model.loc[mask_cardiacas, 'CARDIAC'] = 1

# Visualize a distribuição
print(df_model['CARDIAC'].value_counts())

#### Gerando Df da amostra

In [44]:
codigos_cid = [
    "Q200",  # Tronco arterial comum
    "Q201",  # Ventrículo direito com dupla via de saída
    "Q202",  # Ventrículo esquerdo com dupla via de saída
    "Q203",  # Comunicação ventrículo-atrial discordante
    "Q204",  # Ventrículo com dupla via de entrada
    "Q205",  # Comunicação átrio-ventricular discordante
    "Q206",  # Isomerismo dos apêndices atriais
    "Q208",  # Outras malformações congênitas das câmaras e comunicações cardíacas
    "Q209",  # Malformação congênita não especificada das câmaras e comunicações cardíacas
    "Q210",  # Comunicação interventricular
    "Q211",  # Comunicação interatrial
    "Q212",  # Comunicação atrioventricular
    "Q213",  # Tetralogia de Fallot
    "Q214",  # Comunicação aortopulmonar
    "Q218",  # Outras malformações congênitas dos septos cardíacos
    "Q219",  # Malformação congênita não especificada do septo cardíaco
    "Q220",  # Atresia da valva pulmonar
    "Q221",  # Estenose congênita da valva pulmonar
    "Q222",  # Insuficiência congênita da valva pulmonar
    "Q223",  # Outras malformações congênitas da valva pulmonar
    "Q224",  # Estenose congênita da valva tricúspide
    "Q225",  # Anomalia de Ebstein
    "Q226",  # Síndrome do coração direito hipoplásico
    "Q228",  # Outras malformações congênitas da valva tricúspide
    "Q229",  # Malformação congênita não especificada da valva tricúspide
    "Q230",  # Estenose congênita da valva aórtica
    "Q231",  # Insuficiência congênita da valva aórtica
    "Q232",  # Estenose mitral congênita
    "Q233",  # Insuficiência mitral congênita
    "Q234",  # Síndrome do coração esquerdo hipoplásico
    "Q238",  # Outras malformações congênitas das valvas aórtica e mitral
    "Q239",  # Malformação congênita não especificada das valvas aórtica e mitral
    "Q240",  # Dextrocardia
    "Q241",  # Levocardia
    "Q242",  # Cor triatriatum
    "Q243",  # Estenose do infundíbulo pulmonar
    "Q244",  # Estenose subaórtica congênita
    "Q245",  # Malformações dos vasos coronários
    "Q246",  # Bloqueio congênito do coração
    "Q248",  # Outras malformações congênitas específicas do coração
    "Q249",  # Malformação congênita não especificada do coração
    "Q250",  # Permeabilidade do canal arterial
    "Q251",  # Coartação da aorta
    "Q252",  # Atresia da aorta
    "Q253",  # Estenose da aorta
    "Q254",  # Outras malformações congênitas da aorta
    "Q255",  # Atresia da artéria pulmonar
    "Q256",  # Estenose da artéria pulmonar
    "Q257",  # Outras malformações congênitas da artéria pulmonar
    "Q258",  # Outras malformações congênitas das grandes artérias
    "Q259",  # Malformação congênita não especificada das grandes artérias
    "Q260",  # Estenose congênita da veia cava
    "Q261",  # Persistência da veia cava superior esquerda
    "Q262",  # Comunicação venosa pulmonar anormal total
    "Q263",  # Comunicação venosa pulmonar anormal parcial
    "Q264",  # Comunicação venosa pulmonar anormal não especificada
    "Q265",  # Comunicação venosa portal anormal
    "Q266",  # Fístula entre veia porta e artéria hepática
    "Q268",  # Outras malformações congênitas das grandes veias
    "Q269",  # Malformação congênita não especificada de grande veia
    "Q270",  # Ausência congênita e hipoplasia da artéria umbilical
    "Q271",  # Estenose congênita da artéria renal
    "Q272",  # Outras malformações congênitas da artéria renal
    "Q273",  # Malformação arteriovenosa periférica
    "Q274",  # Ectasia venosa (flebectasia) congênita
    "Q278",  # Outras malformações congênitas específicas do sistema vascular periférico
    "Q279",  # Malformação congênita não especificada do sistema vascular periférico
    "Q280",  # Malformação arteriovenosa de vasos pré-cerebrais
    "Q281",  # Outras malformações dos vasos pré-cerebrais
    "Q282",  # Malformação arteriovenosa dos vasos cerebrais
    "Q283",  # Outras malformações dos vasos cerebrais
    "Q288",  # Outras malformações congênitas específicas do aparelho circulatório
    "Q289",  # Malformação congênita não especificada do aparelho circulatório
    "Q20",  # Malformações congênitas das câmaras e das comunicações cardíacas
    "Q21",  # Malformações congênitas dos septos cardíacos
    "Q22",  # Malformações congênitas das valvas pulmonar e tricúspide
    "Q23",  # Malformações congênitas das valvas mitral e aórtica
    "Q24",  # Outras malformações cardíacas congenítas
    "Q25",  # Anomalias dos grandes vasos arteriais (ex: transposição)
    "Q26",  # Anomalias arteriovenosas cerebrais não classificadas em outra parte.
    "Q27",  # Outras anomalias dos vasos sanguíneos periféricos.
    "Q28"   # Outras malformações congenitas do aparelho circulatório
]

In [ ]:
df_filtrado_variaveis.loc[:, 'CODANOMAL'] = (
    df_filtrado_variaveis['CODANOMAL']
    .astype(str)
    .str.strip()
    .str.upper()
)

# Exclua registros com dados ausentes em variáveis essenciais
df_filtrado_variaveis = df_filtrado_variaveis.dropna(subset=['ESCMAE2010', 'PARTO', 'CODOCUPMAE', 'ESTCIVMAE', 'GRAVIDEZ'])

# Crie uma expressão regular para identificar códigos cardíacos em qualquer posição
regex_cid = '(?:' + '|'.join(codigos_cid) + ')'

# Máscaras para identificar:
# - Registros sem anomalia: CODANOMAL é string vazia
#mask_sem_anomalia = (df_filtrado_variaveis['CODANOMAL'] == '')

# - Registros com malformação cardíaca: CODANOMAL inicia com um dos códigos definidos CASOS
mask_cardiacas = df_filtrado_variaveis['CODANOMAL'].str.contains(regex_cid, na=False)

# Crie o df_model incluindo todos os registros com informação de anomalia
df_model = df_filtrado_variaveis.copy()

# Crie a variável binária 'CARDIAC': 1 para malformação cardíaca, 0 caso contrário
df_model['CARDIAC'] = 0
df_model.loc[mask_cardiacas, 'CARDIAC'] = 1

# Crie a variável 'OTHER_MALF' para outras malformações (não cardíacas):
# Será 1 se houver algum código (não vazio) E esse código NÃO for de malformação cardíaca
#mask_other = (df_model['CODANOMAL'] != '') & (~mask_cardiacas)

#df_model['OTHER_MALF'] = 0
#df_model.loc[mask_other, 'OTHER_MALF'] = 1



# Aplicando as funções no DataFrame
#df_model['CARDIAC'] = df_model['CODANOMAL'].apply(lambda x: 1 if tem_cardiac(x, codigos_cid) else 0)
#df_model['OTHER_MALF'] = df_model['CODANOMAL'].apply(lambda x: 1 if tem_other(x, codigos_cid) else 0)

# Visualize a distribuição das variáveis
print("Distribuição de CARDIAC:")
print(df_model['CARDIAC'].value_counts())

Distribuição de CARDIAC:
CARDIAC
0    5233396
1       6170
Name: count, dtype: int64


In [45]:
df_filtrado_variaveis.loc[:, 'CODANOMAL'] = (
    df_filtrado_variaveis['CODANOMAL']
    .astype(str)
    .str.strip()
    .str.upper()
    .replace(['NAN', 'NaN', 'nan'], '')  # Remove representações de NaN
)

# 3. Função para extrair códigos válidos de uma string concatenada
def extrair_codigos(concatenado):
    # Regex para encontrar códigos CID-10 (ex: Q20, Q21, etc.)
    padrao_cid = r'Q\d{3}'  # Q seguido de 2 dígitos
    return re.findall(padrao_cid, concatenado)

# 4. Aplicar a função para extrair códigos e verificar se são cardíacos
mask_cardiacas = df_filtrado_variaveis['CODANOMAL'].apply(
    lambda s: any(codigo in codigos_cid for codigo in extrair_codigos(s))
)

# 5. Máscara para registros sem anomalia
mask_sem_anomalia = (df_filtrado_variaveis['CODANOMAL'] == '')

# 6. Máscara para outras malformações (não cardíacas e não vazias)
mask_other = (~mask_sem_anomalia)

# 7. Criar as variáveis no DataFrame
df_model = df_filtrado_variaveis.copy()
df_model['CARDIAC'] = 0
df_model.loc[mask_cardiacas, 'CARDIAC'] = 1

df_model['OTHER_MALF'] = 0
df_model.loc[mask_other, 'OTHER_MALF'] = 1

# 8. Visualizar a distribuição
print("Distribuição de CARDIAC:")
print(df_model['CARDIAC'].value_counts())
print("\nDistribuição de OTHER_MALF:")
print(df_model['OTHER_MALF'].value_counts())

Distribuição de CARDIAC:
CARDIAC
0    2460909
1       3110
Name: count, dtype: int64

Distribuição de OTHER_MALF:
OTHER_MALF
0    2441493
1      22526
Name: count, dtype: int64


In [12]:
cases = df_model[df_model['CARDIAC'] == 1]
#controls = df_model[df_model['CARDIAC'] == 0].sample(n = len(cases)*4, random_state=42)
#case_control_df = pd.concat([cases, controls]).sample(frac=1, random_state=42)

In [ ]:
def robust_frequency_matching(df, ratio=4, category_bins=None, return_summary=True):
    """
    Realiza matching por frequência de forma robusta utilizando as variáveis 'IDADEMAE' e 'SEMAGESTAC'.
    
    Parâmetros:
      - df: DataFrame que contenha as colunas:
            'CARDIAC' (1 para caso e 0 para controle),
            'IDADEMAE' e 'SEMAGESTAC'.
      - ratio: número desejado de controles por caso.
      - category_bins: dicionário com os limites dos bins para as variáveis.
           Exemplo: {'IDADEMAE': [10, 20, 30, 40, 50, 60], 'SEMAGESTAC': [28, 32, 37, 42, 50]}
      - return_summary: se True, a função retorna também um DataFrame resumo do matching por grupo.
      
    Retorna:
      - matched_df: DataFrame contendo os casos e os controles pareados.
      - summary_df (opcional): DataFrame com resumo do desempenho do matching em cada grupo.
    """

    df = df.copy()
    
    # Cria as colunas categóricas com base nos bins fornecidos
    if category_bins is not None:
        df['IDADEMAE_CAT'] = pd.cut(df['IDADEMAE'], bins=category_bins['IDADEMAE'], include_lowest=True)
        df['SEMAGESTAC_CAT'] = pd.cut(df['SEMAGESTAC'], bins=category_bins['SEMAGESTAC'], include_lowest=True)
    else:
        df['IDADEMAE_CAT'] = df['IDADEMAE']
        df['SEMAGESTAC_CAT'] = df['SEMAGESTAC']

    
    df = df.dropna(subset=['IDADEMAE_CAT', 'SEMAGESTAC_CAT'])
    
    # Separa casos e controles
    cases = df[df['CARDIAC'] == 1].copy()
    controls = df[df['CARDIAC'] == 0].copy()
    
    grouping_columns = ['IDADEMAE_CAT', 'SEMAGESTAC_CAT']
    # Agrupa os casos pelas combinações de categorias presentes nos dados
    case_groups = cases.groupby(grouping_columns, observed=True)
    
    selected_controls_indices = []
    summary = []  # Para armazenar estatísticas de matching por grupo
    
    for group_keys, group_cases in case_groups:
        # Seleciona os controles que pertencem ao mesmo grupo
        group_controls = controls[
            (controls['IDADEMAE_CAT'] == group_keys[0]) & 
            (controls['SEMAGESTAC_CAT'] == group_keys[1])
        ]
        
        n_cases = len(group_cases)
        required_controls = ratio * n_cases
        
        if len(group_controls) >= required_controls:
            selected = group_controls.sample(n=required_controls, random_state=42)
        else:
            print(f"Aviso: Grupo {group_keys} - necessários {required_controls} controles, disponíveis {len(group_controls)}.")
            selected = group_controls
        
        achieved_ratio = (len(selected) / n_cases) if n_cases > 0 else np.nan
        
        # Registra o resumo do grupo
        summary.append({
            'grupo': group_keys,
            'n_cases': n_cases,
            'required_controls': required_controls,
            'available_controls': len(group_controls),
            'selected_controls': len(selected),
            'achieved_ratio': achieved_ratio
        })
        
        selected_controls_indices.extend(selected.index.tolist())
        # Remove os controles já selecionados para evitar reuso
        controls = controls.drop(selected.index)
    
    matched_controls = df.loc[selected_controls_indices]
    matched_df = pd.concat([cases, matched_controls])
    
    if return_summary:
        summary_df = pd.DataFrame(summary)
        return matched_df, summary_df
    else:
        return matched_df

In [46]:
import pandas as pd
import numpy as np
import warnings
from typing import Optional, Dict, Tuple, Union

def robust_frequency_matching(
    df: pd.DataFrame,
    ratio: int = 4,
    category_bins: Optional[Dict[str, list]] = None,
    return_summary: bool = False,
    random_state: Optional[int] = 42
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Realiza matching por frequência de forma robusta utilizando as variáveis 'IDADEMAE' e 'SEMAGESTAC'.
    
    Parâmetros:
      - df: DataFrame que contenha as colunas:
            'CARDIAC' (1 para caso e 0 para controle),
            'IDADEMAE' e 'SEMAGESTAC'.
      - ratio: Número desejado de controles por caso.
      - category_bins: Dicionário com os limites dos bins para as variáveis.
           Exemplo: {'IDADEMAE': [10, 20, 30, 40, 50, 60], 'SEMAGESTAC': [28, 32, 37, 42, 50]}
      - return_summary: Se True, retorna também um DataFrame resumo do matching por grupo.
      - random_state: Semente para a amostragem aleatória.
      
    Retorna:
      - matched_df: DataFrame contendo os casos e os controles pareados.
      - summary_df (opcional): DataFrame com resumo do desempenho do matching em cada grupo.
    """
    # Verifica se as colunas necessárias estão presentes
    required_cols = {"CARDIAC", "IDADEMAE", "SEMAGESTAC"}
    missing_cols = required_cols - set(df.columns)
    if missing_cols:
        raise ValueError(f"O DataFrame está faltando as seguintes colunas: {missing_cols}")

    df = df.copy()

    # Cria as colunas categóricas com base nos bins fornecidos
    if category_bins is not None:
        idade_bins = category_bins.get("IDADEMAE")
        semage_bins = category_bins.get("SEMAGESTAC")
        if idade_bins is None or semage_bins is None:
            raise ValueError("O dicionário category_bins deve conter as chaves 'IDADEMAE' e 'SEMAGESTAC'")
        df["IDADEMAE_CAT"] = pd.cut(df["IDADEMAE"], bins=idade_bins, include_lowest=True)
        df["SEMAGESTAC_CAT"] = pd.cut(df["SEMAGESTAC"], bins=semage_bins, include_lowest=True)
    else:
        df["IDADEMAE_CAT"] = df["IDADEMAE"]
        df["SEMAGESTAC_CAT"] = df["SEMAGESTAC"]

    # Remove linhas com valores ausentes nas variáveis de categoria
    df.dropna(subset=["IDADEMAE_CAT", "SEMAGESTAC_CAT"], inplace=True)

    # Separa casos e controles
    cases = df[df["CARDIAC"] == 1].copy()
    controls = df[df["CARDIAC"] == 0].copy()

    grouping_columns = ["IDADEMAE_CAT", "SEMAGESTAC_CAT"]

    summary_records = []      # Para armazenar o resumo do matching por grupo
    selected_controls_indices = []  # Índices dos controles selecionados

    # Agrupa os casos pelas combinações de categorias
    case_groups = cases.groupby(grouping_columns, observed=True)

    for group_keys, group_cases in case_groups:
        n_cases = len(group_cases)
        required_controls = ratio * n_cases

        # Seleciona os controles que pertencem ao mesmo grupo
        group_mask = (
            (controls["IDADEMAE_CAT"] == group_keys[0]) &
            (controls["SEMAGESTAC_CAT"] == group_keys[1])
        )
        group_controls = controls.loc[group_mask]

        if len(group_controls) >= required_controls:
            selected = group_controls.sample(n=required_controls, random_state=random_state)
        else:
            warnings.warn(
                f"Aviso: Grupo {group_keys} - necessários {required_controls} controles, disponíveis {len(group_controls)}."
            )
            selected = group_controls

        achieved_ratio = len(selected) / n_cases if n_cases > 0 else np.nan

        # Registra as estatísticas do grupo
        summary_records.append({
            "grupo": group_keys,
            "n_cases": n_cases,
            "required_controls": required_controls,
            "available_controls": len(group_controls),
            "selected_controls": len(selected),
            "achieved_ratio": achieved_ratio
        })

        selected_controls_indices.extend(selected.index.tolist())
        # Remove os controles já selecionados para evitar reuso
        controls = controls.drop(selected.index)

    matched_controls = df.loc[selected_controls_indices]
    matched_df = pd.concat([cases, matched_controls]).sort_index()

    if return_summary:
        summary_df = pd.DataFrame(summary_records)
        return matched_df, summary_df
    else:
        return matched_df


In [47]:
age_bins = [10,20,30,40,50,60]
gestation_bins = [22,24,27,31,36,41,60]
category_bins = {'IDADEMAE': age_bins, 'SEMAGESTAC': gestation_bins}

In [48]:
matched_df_= robust_frequency_matching(df_model, ratio=4, category_bins=category_bins)

In [49]:
print(matched_df_['CARDIAC'].value_counts())

CARDIAC
0    12416
1     3104
Name: count, dtype: int64


In [50]:
matched_df_.to_csv("amostra/data_BR_Match.csv", encoding='utf-8')